In [11]:
import pandas as pd
import numpy as np
import requests
import datetime as dt


page = 1

result_list = list()

while True:

    url = f"https://apewisdom.io/api/v1.0/filter/all-stocks/page/{page}"

    response = requests.get(url)

    for result in response.json()["results"]:
        result_list.append(result)

    page += 1

    if not response.json()["results"]:
        break

df = pd.DataFrame(result_list)

# Transformations
df.loc[df["rank_24h_ago"] == "0", "rank_24h_ago"] = None
df.loc[df["mentions_24h_ago"].isnull(), "mentions_24h_ago"] = "0"
df.loc[df["mentions"].isnull(), "mentions"] = "0"
df.loc[df["upvotes"].isnull(), "upvotes"] = "0"


df = df.astype({"rank": "int32",
        "ticker": "object",
        "name": "object",
        "mentions": "int32",
        "upvotes": "int32",
        "rank_24h_ago": "object",
        "mentions_24h_ago": "int32"})

df


,rank,ticker,name,mentions,upvotes,rank_24h_ago,mentions_24h_ago
0,1,TSLA,Tesla,967,32392,5,181
1,2,SPY,SPDR S&amp;P 500 ETF Trust,312,987,3,344
2,3,AI,C3 AI,278,1986,2,371
3,4,NVDA,NVIDIA,245,756,4,216
4,5,FRC,First Republic Bank,154,1083,7,106
...,...,...,...,...,...,...,...
1104,1105,TTM,Tata Motors,1,3,194,4
1105,1106,CE,Celanese,1,0,1055,0
1106,1107,CTRA,Coterra Energy,1,0,None,0
1107,1108,OPK,OPKO Health,1,2,1057,1


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1108 entries, 0 to 1107
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   rank              1108 non-null   int32 
 1   ticker            1108 non-null   object
 2   name              1108 non-null   object
 3   mentions          1108 non-null   int32 
 4   upvotes           1108 non-null   int32 
 5   rank_24h_ago      956 non-null    object
 6   mentions_24h_ago  1108 non-null   int32 
dtypes: int32(4), object(3)
memory usage: 43.4+ KB


In [13]:
sum([3,3,4,4,7,5,1,9,6,8,12,29,71,58,36,29,22,19,21,14,9,7,8,6,8,7,3,17,13,22,16,11,10,9,11,9,6,7,1,1,2,8,4,2,2,4,3,2,2,6,4,2,2,1,3,2,3,3,1,1,2,2,0,1,4,0,0,0,0,2,0,1,0,2,3,0,0,2,0,1,2,1,4,1,0,1,6,1,2,1,3,2,2,0,1,1,5,2,3,2,2,3,3,5,8,5,6,4,5,3,7,6,12,9,14,7,14,12,8,6,6,4,2,5,3,8,7,6,5,4,13,5,12,19,22,7,6,5,9,5,9,4,1,2])

967

In [14]:
from bs4 import BeautifulSoup
import re

url = "https://apewisdom.io/stocks/TSLA/"

response = requests.get(url)

text = response.text

In [17]:
soup = BeautifulSoup(text, 'html.parser')

In [19]:
print(soup.prettify())

<html lang="en-US">
 <head>
  <title>
   TSLA - Tesla, Reddit sentiment and mention history
  </title>
  <meta charset="utf-8"/>
  <meta content="INDEX,FOLLOW" name="robots"/>
  <meta content="During the last 24h, TSLA was mentioned 930 times on r/wallstreetbets by a total of 460 different users" name="description"/>
  <meta content="en-us" http-equiv="content-language"/>
  <meta content="TSLA - Tesla, Reddit sentiment and mention history" property="og:title"/>
  <meta content="website" property="og:type">
   <meta content="During the last 24h, TSLA was mentioned 930 times on r/wallstreetbets by a total of 460 different users" property="og:description"/>
   <meta content="https://apewisdom.io/apewisdom-media.png" property="og:image"/>
   <link href="https://apewisdom.io/stocks/TSLA/" rel="canonical"/>
   <meta content="https://apewisdom.io/stocks/TSLA/" property="og:url">
    <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
    <link href="/apple-touch-icon.png" 

In [20]:
script = soup.findAll('script')

In [24]:
script[5].string

'\nChart.defaults.LineWithLine = Chart.defaults.line;\nChart.controllers.LineWithLine = Chart.controllers.line.extend({\n   draw: function(ease) {\n      Chart.controllers.line.prototype.draw.call(this, ease);\n\n      if (this.chart.tooltip._active && this.chart.tooltip._active.length) {\n         var activePoint = this.chart.tooltip._active[0],\n         ctx = this.chart.ctx,\n         x = activePoint.tooltipPosition().x,\n         topY = this.chart.legend.bottom,\n         bottomY = this.chart.chartArea.bottom;\n\n         // draw line\n         ctx.save();\n         ctx.beginPath();\n         ctx.moveTo(x, topY);\n         ctx.lineTo(x, bottomY);\n         ctx.lineWidth = 2;\n         ctx.strokeStyle = \'#aaaaaa\';\n         ctx.stroke();\n         ctx.restore();\n      }\n   }\n});\n\n\n\nnew Chart(document.getElementById("chart-overall-mentions"), {\n  type: \'LineWithLine\',\n  data: {\n    labels: [1681930503000,1681931103000,1681931703000,1681932303000,1681932903000,1681933503

In [55]:
import re
import datetime as dt

In [68]:
time = re.findall("labels:.*],", script[5].string)[0][9:-2].split(",")
number_of_mentions = re.findall("data:.*],", script[5].string)[0][7:-2].split(",")

In [69]:
len(number_of_mentions) == len(time)

True

In [70]:
len(number_of_mentions)

144

In [71]:
len(time)

144

In [57]:
datetimes = [dt.datetime.fromtimestamp(int(timestamp)) for timestamp in time]

OSError: [Errno 22] Invalid argument

In [64]:
dt.datetime.fromtimestamp(int(time[0][:-3])).strftime("%m/%d/%Y, %H:%M:%S")

'04/19/2023, 20:55:03'

In [60]:
int(time[0])


1681930503000

In [65]:
dt.datetime.fromtimestamp(int(time[-1][:-3])).strftime("%m/%d/%Y, %H:%M:%S")

'04/20/2023, 20:45:03'

In [66]:
number_of_mentions[-1]

'6'

In [67]:
number_of_mentions[0]

'2'